In [143]:
from db import ohlcv_db, posts_db
from typing import List,Set,Iterable
import pymongo
import pprint

In [144]:
def get_top_pairs(max_num:int=100) -> List:
    """
        Function to return at most the top max_num distinct pairs by compound volume from ohlcv db
    """
    ohlcv_documents = ohlcv_db.find({},{'_id':False}).sort('volume', pymongo.DESCENDING)

    top_100_pairs_by_volume_set = set()
    top_100_pairs_by_volume = list()
    for item in ohlcv_documents:

        pair = f"{item['pair_symbol'].upper()}-{item['pair_base'].upper()}"
        if pair not in top_100_pairs_by_volume_set:
            top_100_pairs_by_volume_set.add(pair)
            top_100_pairs_by_volume.append(pair)
            
        if len(top_100_pairs_by_volume) == max_num:
            break
    return top_100_pairs_by_volume

In [145]:
def get_oldest_posted_pairs(top_pairs:List,max_num:int=50) -> Set:

    posted_pairs_among_top = posts_db.find({'pair' : {'$in':top_pairs}}).sort('time', pymongo.ASCENDING).distinct('pair')
    
    posted_pairs_set_among_top = set()

    # At most top max_num distinct oldest posted pairs
    for pair in posted_pairs_among_top:
        if pair not in posted_pairs_set_among_top:
            posted_pairs_set_among_top.add(pair)
        if len(posted_pairs_set_among_top) == max_num:
            break
    return posted_pairs_set_among_top

In [146]:
def get_last_posted_pair() -> str:
    
    document = posts_db.find({}).sort('time', pymongo.DESCENDING).limit(1)

    return document[0].get('pair')

In [147]:
def get_pair_to_post(top_pairs:List,oldest_posted_pairs:Iterable = None) -> str:
    
    
    if (oldest_posted_pairs is None) or len(oldest_posted_pairs) == 0:
        # the first pair by volume
        return top_pairs[0]

    last_posted = get_last_posted_pair()
    for pair in top_pairs:
        if pair in oldest_posted_pairs and not pair == last_posted:
            # Choose the first pair by volume among those oldest posted pairs
            return pair
    else:
        # Or the first pair by volume
        return top_pairs[0]

In [148]:
# top_100_pairs_by_volume = get_top_pairs(max_num=100)
# oldest_posted_pair = get_oldest_posted_pairs(top_pairs=top_100_pairs_by_volume,max_num=5)
# get_pair_to_post(top_100_pairs_by_volume,oldest_posted_pair)

In [149]:
top = ['A','B','C']
assert('A' == get_pair_to_post(top,oldest_posted_pairs={}))
assert('B' == get_pair_to_post(top,oldest_posted_pairs={'B'}))

In [150]:
posts_documents_for_top_pairs = posts_db.find({}).sort('time', pymongo.ASCENDING)
for post in posts_documents_for_top_pairs:
    pprint.pprint(post)

{'_id': ObjectId('639532ca0d693dfae1b8b0b4'),
 'pair': 'LTC-USD',
 'time': datetime.datetime(2022, 12, 12, 12, 55),
 'tweet_text': 'Top Market Venues for LTC-USD:\n'
               'Coinbase: 56.15%\n'
               'Kraken: 15.95%\n'
               'Binance-us: 11.64%\n'
               'Bitfinex: 8.54%\n'
               'Crypto-com: 3.09%\n'
               'Others: 4.64%'}
{'_id': ObjectId('639532ca0d693dfae1b8b0ae'),
 'pair': 'ETH-USD',
 'time': datetime.datetime(2022, 12, 12, 13, 34),
 'tweet_text': 'Top Market Venues for ETH-USD:\n'
               'Coinbase: 61.2%\n'
               'Kraken: 10.75%\n'
               'Bitfinex: 9.08%\n'
               'Crypto-com: 8.51%\n'
               'Gemini: 4.92%\n'
               'Others: 5.53%'}
{'_id': ObjectId('639532ca0d693dfae1b8b0b2'),
 'pair': 'LINK-USD',
 'time': datetime.datetime(2022, 12, 12, 14, 43),
 'tweet_text': 'Top Market Venues for LINK-USD:\n'
               'Coinbase: 63.44%\n'
               'Binance-us: 17.34%\n'
        

In [151]:

def get_message_dict(pair:str=None,pair_symbol:str=None,pair_base:str=None) -> dict:
    """
        Function to return a dictionary of marketVenue:percentage of volume for a given pair
    """
    if pair is None and (pair_symbol is None or pair_base is None):
        raise ValueError("Either pair or pair_symbol and pair_base must be provided")

    if pair is not None:
        try:
            pair_symbol = pair.split('-')[0].lower()
            pair_base = pair.split('-')[1].lower()
        except:
            raise ValueError("Invalid pair provided, must be in the format 'SYMBOL-BASE")
            
    ohlcv_documents_for_pair = ohlcv_db.find({'pair_symbol':pair_symbol,'pair_base':pair_base}).sort([
        ('timestamp', pymongo.DESCENDING),
        ])

    latest_date = None
    message_dict = {
        'others':0
    }
    for item in ohlcv_documents_for_pair:
        
        if latest_date is None:
            latest_date = item['timestamp']
        if item['timestamp'] == latest_date:
            # pprint.pprint(item)
            if item['marketVenue'] not in message_dict:
                if len(message_dict) <= 5:
                    # key for top 5 marketVenues
                    message_dict[item['marketVenue']] = float(item['volume'])
                else:
                    # store the rest in 'others'
                    message_dict['others'] += float(item['volume'])
                continue    
            message_dict[item['marketVenue']] += float(item['volume'])
        else:
            break
    total_volume = sum(message_dict.values())
    message_dict = {k:round((v/total_volume)*100,2) for k,v in message_dict.items()}
    return message_dict

In [159]:
def compose_message(pair:str,message_dict:dict) -> str:
    """
        Function to compose a message for a given pair and message_dict
    """
    message = f"Top 5 marketVenues for {pair} are:\n"
    for marketVenue,percentage in sorted(message_dict.items()):
        message += f"{marketVenue.capitalize()}: {percentage}%\n"
    return message

In [160]:
pair = 'ETH-USD'
message_dict = get_message_dict(pair=pair)
message = compose_message(pair,message_dict)
print(message)

Top 5 marketVenues for ETH-USD are:
Binance: 79.85%
Gateio: 0.0%
Huobi: 3.85%
Kraken: 3.41%
Okx: 12.28%
Others: 0.62%



In [ ]:
a = dict()
a['a'] = 1
print(dir(a))

['__class__', '__class_getitem__', '__contains__', '__delattr__', '__delitem__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__ior__', '__iter__', '__le__', '__len__', '__lt__', '__ne__', '__new__', '__or__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__ror__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', 'clear', 'copy', 'fromkeys', 'get', 'items', 'keys', 'pop', 'popitem', 'setdefault', 'update', 'values']
